In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn import metrics

In [2]:
df = pd.read_csv('Twitter_stock_final_dataset_new.csv')
df.head()

Year  Month  Day  StockName  Positive  Negative  Neutral  Total Tweets  \
0  2020      1    1      apple        10         2        8            20   
1  2020      1    1  microsoft         9         0       11            20   
2  2020      1    1      tesla        17         3        3            23   
3  2020      1    1     nvidia         1         0        0             1   
4  2020      1    1     paypal         1         0        1             2   

      Close       Volume      Open      High       Low Day_of_week  
0   75.0875  135647440.0   74.0600   75.1500   73.7975   Wednesday  
1  160.6200   22634550.0  158.7800  160.7300  158.3300   Wednesday  
2   86.0520   47791930.0   84.9000   86.1391   84.3420   Wednesday  
3   59.9775   23767876.0   59.6875   59.9775   59.1800   Wednesday  
4  110.7500    7419283.0  109.4700  111.2100  109.2750   Wednesday

In [3]:
df2 = df

In [4]:
Date = ['Year', 'Month','Day']
df2.sort_values(by = Date, inplace = True, ascending = True)

In [5]:
# seperate independent vs dependent vriables

#cols = ['Close', 'High', 'Low', 'Volume','Open','Day_of_week']
X_reg3 = df2.drop('Close', axis = 1).values
y_reg3 = df2.Close.values

In [6]:
df_X = pd.DataFrame(X_reg3)#, columns=['Column1', 'Column2'] 
df_X.head()

0  1  2          3   4  5   6   7            8        9        10  \
0  2020  1  1      apple  10  2   8  20  135647440.0    74.06    75.15   
1  2020  1  1  microsoft   9  0  11  20   22634550.0   158.78   160.73   
2  2020  1  1      tesla  17  3   3  23   47791930.0     84.9  86.1391   
3  2020  1  1     nvidia   1  0   0   1   23767876.0  59.6875  59.9775   
4  2020  1  1     paypal   1  0   1   2    7419283.0   109.47   111.21   

        11         12  
0  73.7975  Wednesday  
1   158.33  Wednesday  
2   84.342  Wednesday  
3    59.18  Wednesday  
4  109.275  Wednesday

In [7]:
#LABEL ENCODE
# Encoding the Independent or ohject variable Day_of_week
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

regEnc = LabelEncoder()

X_reg3[:, -1] = regEnc.fit_transform(X_reg3[:, -1]) #Day_of_week

X_reg3[:, 3] = regEnc.fit_transform(X_reg3[:, 3]) #stockName

In [8]:
#Split the new data into training and testing sets

Xtrain_set3, Xtest_set3= np.split(X_reg3, [int(0.8 * len(X_reg3))])
ytrain_set3, ytest_set3= np.split(y_reg3, [int(0.8 * len(y_reg3))])

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor() 


params = {'learning_rate':[0.01,0.03,0.05,0.1],
          'n_estimators':[5,10,15,20,50,70,100],
          'max_depth':[3,5,10]} 

cross_val = RandomizedSearchCV(gbr, param_distributions=params, n_iter=10,cv=5,n_jobs=-1, verbose=10)
cross_val.fit(Xtrain_set3,ytrain_set3)

print('='*100)
print('The Best Parameters are : ',cross_val.best_params_)
print('='*100) 

xgb_best = cross_val.best_estimator_ 
xgb_best.fit(Xtrain_set3,ytrain_set3)

y_Lpred3 = xgb_best.predict(Xtest_set3)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
The Best Parameters are :  {'n_estimators': 100, 'max_depth': 10, 'learning_rate': 0.1}


In [10]:
#Scores

print('GBDT SCORES----')
print()

def mape(actual, predicted):
    mape = np.mean(np.abs((actual - predicted)/actual))*100
    return mape

#mean absolute percentage error
print('mean absolute percentage error: ', mape(ytest_set3, y_Lpred3))

mse =np.square(np.subtract(ytest_set3, y_Lpred3)).mean()
rmse = math.sqrt(mse)
print('rmse: ', rmse)

#Accuracy
print('Mean absolute error: ', metrics.mean_absolute_error(ytest_set3, y_Lpred3))

#r2_score
print('The r2_score: ', metrics.r2_score(ytest_set3, y_Lpred3))

GBDT SCORES----

mean absolute percentage error:  0.918666661586828
rmse:  5.603250183962114
Mean absolute error:  3.382022620823264
The r2_score:  0.9991328459500315


In [11]:
result = pd.DataFrame({'Actual': ytest_set3.flatten(), 'Predicted': y_Lpred3.flatten()})
result

Actual   Predicted
0    126.543333  126.639152
1    248.910000  248.806971
2    597.920000  589.335878
3    154.052500  154.994882
4    254.355000  250.756431
..          ...         ...
592  730.170000  733.628978
593  211.130000  209.893539
594  269.910000  269.824072
595  212.460000  211.638406
596  269.490000  268.031642

[597 rows x 2 columns]